In [1]:
import os
import json
import pandas as pd
from multiprocessing import Pool
from PIL import Image

In [2]:
input_path = '../datasets/concap/download_tool/'
image_path = '../datasets/concap/images/'

In [3]:
df1 = pd.read_csv(
    os.path.join(input_path, 'Validation_GCC-1.1.0-Validation.tsv'),
    sep='\t', 
    names=["caption","url"]
)

In [4]:
df2 = pd.read_csv(
    os.path.join(input_path, 'downloaded_validation_report.tsv'),
    sep='\t', 
    names=["fname", "split", "ftype", "fsize", "response", "url"]
)

In [5]:
df2['caption'] = df1['caption']

In [6]:
df2['ftype'].unique()

array(['image/jpeg', nan, 'image/gif', 'text/html', 'image/png',
       'inode/x-empty', 'text/plain', 'text/xml'], dtype=object)

In [7]:
def process(row):
    if row['response'] == 200 and 'image' in row['ftype']:
        try:
            img = Image.open(os.path.join(image_path, row['fname'])).convert('RGB')
            row['width'] = img.size[0]
            row['height'] = img.size[1]
        except Exception as ex: 
            row['exception'] = str(ex)
    return row

In [8]:
items = []
with Pool(32) as pool:
    for i, row in enumerate(pool.imap_unordered(
        process, df2.to_dict(orient='records'), 10)):
        if 'width' in row:
            items.append(row)
        elif 'exception' in row:
            print(row)
        if i % 10000 == 0 and i > 0: 
            print(f'{int(i/10000)} out of {int(df2.index.size/10000)}')        

/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


1 out of 1
{'fname': 'validation/3044488379', 'split': 'validation', 'ftype': 'image/jpeg', 'fsize': 48826.0, 'response': 200, 'url': 'http://www.themalaymailonline.com/uploads/articles/2016-04/subaru_forester_1504.jpg', 'caption': 'automobile model -- picture courtesy', 'exception': 'image file is truncated (2 bytes not processed)'}


In [9]:
len(items)

13565

In [10]:
with open('../datasets/concap/validation.json', 'w') as fout:
    json.dump(items, fout)

In [11]:
df1 = pd.read_csv(
    os.path.join(input_path, 'Train_GCC-training.tsv'),
    sep='\t', 
    names=["caption","url"]
)

In [12]:
df2 = pd.read_csv(
    os.path.join(input_path, 'downloaded_training_report.tsv'),
    sep='\t', 
    names=["fname", "split", "ftype", "fsize", "response", "url"]
)

In [13]:
df2['caption'] = df1['caption']

In [14]:
items = []
with Pool(32) as pool:
    for i, row in enumerate(pool.imap_unordered(
        process, df2.to_dict(orient='records'), 10)):
        if 'width' in row:
            items.append(row)
        elif 'exception' in row:
            print(row)
        if i % 100000 == 0 and i > 0: 
            print(f'{int(i/100000)} out of {int(df2.index.size/100000)}')        

/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


1 out of 331


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (93950400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:766: UserWarning: Possibly corrupt EXIF data.  Expecting to read 35 bytes but only got 34. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag)


{'fname': 'training/1425666352', 'split': 'training', 'ftype': 'image/jpeg', 'fsize': 1736374.0, 'response': 200, 'url': 'https://www.amherst.edu/system/files/styles/original/private/media/IMG_6564.JPG', 'caption': 'a view of the medieval market', 'exception': 'image file is truncated (51 bytes not processed)'}
{'fname': 'training/2968766090', 'split': 'training', 'ftype': 'image/jpeg', 'fsize': 144577.0, 'response': 200, 'url': 'http://saltwaterlife.co.uk/wp-content/uploads/2013/09/IMG_9549-001.jpg', 'caption': 'a large white shark , turns on the approach to the camera', 'exception': 'image file is truncated (13 bytes not processed)'}
{'fname': 'training/978654963', 'split': 'training', 'ftype': 'image/jpeg', 'fsize': 1835008.0, 'response': 200, 'url': 'http://inktank.fi/wp-content/uploads/2017/07/finnish-midsummer-sunset-young-man-throwing-rock-lake-landscape-markus-watkins-10.jpg', 'caption': 'young man throwing a stone into a lake , during sunset .', 'exception': 'image file is tru

/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


...

{'fname': 'training/432150964', 'split': 'training', 'ftype': 'image/jpeg', 'fsize': 16691.0, 'response': 200, 'url': 'http://anacortes.org/wp-content/uploads/2015/03/orca-1-632x444.jpg', 'caption': 'biological species in the waters around a city', 'exception': 'image file is truncated (1 bytes not processed)'}
{'fname': 'training/1500833971', 'split': 'training', 'ftype': 'image/jpeg', 'fsize': 149579.0, 'response': 200, 'url': 'http://morganhilllife.com/wp-content/uploads/2016/03/Coyote-Ridge-March-18-21-1.jpg', 'caption': 'years in the making will be permanently preserved', 'exception': 'image file is truncated (10 bytes not processed)'}
{'fname': 'training/1001986659', 'split': 'training', 'ftype': 'image/jpeg', 'fsize': 39824.0, 'response': 200, 'url': 'https://thumb10.shutterstock.com/display_pic_with_logo/641824/492631483/stock-photo-boy-and-his-dog-playing-in-a-pile-of-autumn-leaves-492631483.jpg', 'caption': 'boy and his dog playing in a pile of autumn leaves', 'exception': 'i

/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


{'fname': 'training/3965891445', 'split': 'training', 'ftype': 'image/jpeg', 'fsize': 32768.0, 'response': 200, 'url': 'https://thumb7.shutterstock.com/display_pic_with_logo/162468982/757093999/stock-vector-card-invitation-cover-template-design-line-art-background-abstract-geometric-pattern-with-place-757093999.jpg', 'caption': 'card , invitation , cover template design , line art background .', 'exception': 'image file is truncated (1 bytes not processed)'}


/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:766: UserWarning: Possibly corrupt EXIF data.  Expecting to read 11 bytes but only got 10. Skipping tag 42037
  " Skipping tag %s" % (size, len(data), tag)
/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


33 out of 331


In [15]:
len(items)

2749293

In [16]:
with open('../datasets/concap/train.json', 'w') as fout:
    json.dump(items, fout)